# Linear Model Using Pytorch

With the blue cube to the left, load:

python/3.8.5-fasrc01

cuda/11.1.0-fasrc01

In [97]:
# import os

# os.chdir("/n/dominici_nsaph_l3/Lab/projects/heat-alerts_mortality_RL")

! python ~/Q_prep.py

In [77]:
## Set up the model

class DQN(nn.Module):
    def __init__(self, n_col) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_col, 32),
            nn.SiLU(),
            nn.Linear(32, 32),
            nn.SiLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.net(x)

In [78]:
## Methods to evaluate the model

def eval_Q_double(Q_model, Qtgt_model, S, over = None): 

    Q = Q_model(S) # n x 2
    Qtgt = Qtgt_model(S) # n x 2
    best_action = Q.argmax(axis=1).view(-1, 1)
    best_Q = torch.gather(Qtgt, 1, best_action).view(-1) # n 

    if over is not None:
        final_Q = torch.where(over, Qtgt[:,0], best_Q) # n
        return final_Q

    return best_Q

In [ ]:
## Set up dataloader
data = [S, A, R, S_1, ep_end, over]

dev = "cpu"
tensors = [torch.from_numpy(v.to_numpy()).to(dev) for v in data]

DS = TensorDataset(*tensors)

torch.manual_seed(321)

DL = DataLoader(DS, batch_size = 256, shuffle = True, pin_memory = True)

In [1]:
## Initialize the model

state_dim = S.shape[1]
num_episodes = n_counties*n_years
# print(num_episodes)
n = num_episodes*(n_days-1)

model = DQN(state_dim)
model = model.to(dev)

# optimizer = optim.Adam(model.parameters(), lr = 0.0001, betas=(0.25, 0.99))
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.25)

update_tgt_every = 50
print_every = 10
tgt_model = deepcopy(model)


NameError: name 'S' is not defined

In [ ]:
## Train

epochs = 2000

epoch_loss_means = []
for e in range(epochs):
    l = []
    iter = 1
    for s, a, r, s1, e, over in tqdm(DL, disable=True):
        with torch.no_grad():
          # update with tgt copy model as in Double DQN paper
          target = r + gamma*(1-e.float())*eval_Q_double(model, tgt_model, s1, over)

        optimizer.zero_grad()

        output = model(s) # n x 2
        Q = torch.gather(output, 1, a.view(-1, 1)).view(-1)

        # loss = F.mse_loss(Q, target)
        loss = F.smooth_l1_loss(Q, target) # huber loss as in DQN paper

        loss.backward()
        optimizer.step()
        l.append(loss.item())
        
        iter+=1
        if iter == 100:
            break

    epoch_loss = np.mean(l)
    epoch_loss_means.append(epoch_loss)
    if e % print_every == 0:
        print(f"Epoch: {e}, average loss: {epoch_loss:.4f}")

    if e % update_tgt_every == 0:
        tgt_model = deepcopy(model)
  